In [2]:
# Import all packages used
import pandas as pd
import numpy as np
import spacy as sp
import string
import en_core_web_sm
import nltk as nl
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn import svm

from sklearn.ensemble import RandomForestClassifier


from sklearn.naive_bayes import BernoulliNB

model = svm.SVC()
param = {'kernel': ['linear','rbf']}

lemm_obj = sp.load('en_core_web_lg')
lemm_obj.max_length = 1500000
ps = nl.PorterStemmer()

vect_cnt = CountVectorizer(analyzer='word',min_df=1,stop_words='english',token_pattern='[a-zA-Z0-9]{1,}')
tfidf_vect = TfidfVectorizer(analyzer = 'word', min_df = 1, stop_words = 'english',vocabulary=20000, token_pattern = '[a-zA-Z0-9]{1,}')

allowed_postags = ['NOUN','ADJ','VERB','ADV']

import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import IPython

In [3]:
gen_df_encode = pd.read_pickle("D:\Machine Learning\Vamsi Thesis\Code\Results\gen_df_encode.pkl")

In [4]:
fullMovieData = pd.read_pickle(r"D:\Machine Learning\Vamsi Thesis\Code\Results\fullMovieData.pkl")
# List of genres
g_list = fullMovieData.apply(lambda x: x['genres'].split('|'),axis = 1)
g_list=g_list.to_list()
g_list=list(set(sum(g_list,[])))
g_list

['Horror',
 'War',
 'Romance',
 'Action',
 'Musical',
 'Animation',
 'IMAX',
 'Drama',
 'Fantasy',
 'Crime',
 'Sci-Fi',
 'Western',
 'Adventure',
 'Film-Noir',
 'Mystery',
 'Documentary',
 'Comedy',
 'Thriller',
 'Children']

In [5]:
Final_data=pd.DataFrame(columns=["Genre","Actual Positives","Actual Negatives","Vectorizer","Model","Test_Precision","Test_Recall","Test_Fscore","Precision","Recall","Fscore"])
Final_data.shape
#Master function SVM RF Naive Bayes
def prediction(df,text, genre, vectorizer, model, param):
    result=[]
    print("Genre for prediction: ", genre)
    result.append(genre)
    
    #Sampling
    df_model_0 = df[df[genre] == 0]
    df_model_1 = df[df[genre] == 1]
    print("Number of 0s: ", df_model_0.shape[0])
    print("Number of 1s: ", df_model_1.shape[0])
    result.append(df_model_0.shape[0])
    result.append(df_model_1.shape[0]) 
    
    sample_size = 2*df_model_1.shape[0]
    if sample_size > len(df_model_0) : sample_size = len(df_model_0)
    df_model_s=pd.concat([df_model_1,df_model_0.sample(sample_size)])
    print("Shape of the Sampled DataFrame: ", df_model_s.shape)
    #result.append(df_model_s.shape[0])
    
    #Train-Test Split
    X_df = df_model_s[text]
    y_df = df_model_s[genre]
    X_train,X_test,y_train,y_test = train_test_split(X_df, y_df, test_size = 0.25, random_state = 1,
                                                     shuffle = True)
    
    #Vectorization
    print("\n")
    print("Vectorization Start")
    vect = vectorizer(analyzer = 'word', min_df = 1, stop_words = 'english', token_pattern = '[a-zA-Z]{2,}')
    vect_fit = vect.fit(X_train)
    result.append(vectorizer)
    
    X_train_vec = pd.DataFrame(vect_fit.transform(X_train).toarray())
    X_train_vec.columns = vect_fit.get_feature_names()
    
    X_test_vec = pd.DataFrame(vect_fit.transform(X_test).toarray())
    X_test_vec.columns = vect_fit.get_feature_names()
    print("Vectorization End")
    
    #Model
    print("\n")
    print("Model Building Start")
    model = model
    param = param
    result.append(model)
    
    gs = GridSearchCV(model, param, cv = 5,n_jobs=2)
    gs_fit = gs.fit(X_train_vec, y_train)
    
    test_pred_cnt = pd.DataFrame(gs.predict(X_test_vec))
    train_pred_cnt = pd.DataFrame(gs.predict(X_train_vec))
    print("Model Building End")

    # Accuracy metrics

    #Test Accuracy
    print("\n")
    print("Sample data Test Set Confusion Matrix: ")
    print(confusion_matrix(y_test, test_pred_cnt))
    precision, recall, fscore, train_support = score(y_test, test_pred_cnt, pos_label = 1, average = 'binary')
    print("Sample data Test Set Precision: ", precision)
    print("Sample data Test Set Recall: ", recall)
    print("Sample data Test Set FScore: ", fscore)
    print("\n")
    result.extend([precision, recall, fscore])

    # Train Accuracy
    print("Sample data Train Set Confusion Matrix: ")
    print(confusion_matrix(y_train, train_pred_cnt))
    precision, recall, fscore, train_support = score(y_train, train_pred_cnt, pos_label = 1, average = 'binary')
    print("Sample data Train Set (Precision, Recall, Fscore) :",precision, recall, fscore)
    print("\n")

    #Accuracy on full Data
    X_df_full = df[text]
    y_df_full = df[genre]

    # vectorizer
    X_full_tf = pd.DataFrame(vect_fit.transform(X_df_full).toarray())

    print("Probability Evaluation and Concat Start")
    full_pred_cnt = pd.DataFrame(gs.predict(X_full_tf))
    full_pred_prob = pd.DataFrame(gs.predict_proba(X_full_tf))
    df = pd.concat([df,full_pred_prob], axis = 1)
    df.rename(columns ={1:genre+'_prob'},inplace=True)
    df.drop([0],axis=1,inplace=True)
    print("Probability Evaluation and Concat End")
    
    print("\n")
    print("Full Dataset Confusion Matrix " , model)
    print(confusion_matrix(y_df_full, full_pred_cnt))
    precision, recall, fscore, train_support = score(y_df_full, full_pred_cnt, pos_label = 1, average = 'binary')
    print("Full Dataset Precision: ", precision)
    print("Full Dataset Recall: ", recall)
    print("Full Dataset FScore: ", fscore)
    print("\n")
    result.extend([precision, recall, fscore])
    
    print("Appending results to Final_data Start")
    Final_data.loc[len(Final_data)]=result
    print("Appending results to Final_data End")
    return df

In [7]:
start = time.time()
df_test=gen_df_encode
for i in g_list:    
    df_test  = prediction(df_test,'tt_lemm',i,TfidfVectorizer, BernoulliNB(),  {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0]})
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\NB_Probability_Distribution_plots.pkl')
Final_data.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\Result_NB.pkl')
end = time.time()
total_time = end-start

print("Total Time taken:" + str(end-start))

Genre for prediction:  Horror
Number of 0s:  9456
Number of 1s:  946
Shape of the Sampled DataFrame:  (2838, 29)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[378  98]
 [ 76 158]]
Sample data Test Set Precision:  0.6171875
Sample data Test Set Recall:  0.6752136752136753
Sample data Test Set FScore:  0.6448979591836735


Sample data Train Set Confusion Matrix: 
[[1278  138]
 [   6  706]]
Sample data Train Set (Precision, Recall, Fscore) : 0.8364928909952607 0.9915730337078652 0.9074550128534705


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  BernoulliNB()
[[7766 1690]
 [  82  864]]
Full Dataset Precision:  0.33829287392325763
Full Dataset Recall:  0.9133192389006343
Full Dataset FScore:  0.4937142857142857


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  War
Number of 0s:  10100
Number of 1s:  

Full Dataset Confusion Matrix  BernoulliNB()
[[9122  691]
 [ 123  466]]
Full Dataset Precision:  0.40276577355229043
Full Dataset Recall:  0.7911714770797963
Full Dataset FScore:  0.5337915234822451


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Crime
Number of 0s:  9176
Number of 1s:  1226
Shape of the Sampled DataFrame:  (3678, 38)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[543  53]
 [202 122]]
Sample data Test Set Precision:  0.6971428571428572
Sample data Test Set Recall:  0.3765432098765432
Sample data Test Set FScore:  0.4889779559118237


Sample data Train Set Confusion Matrix: 
[[1826   30]
 [ 144  758]]
Sample data Train Set (Precision, Recall, Fscore) : 0.9619289340101523 0.8403547671840355 0.8970414201183433


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  BernoulliNB()
[[8496  6

C:\Users\ragha\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sample data Test Set Precision:  0.0
Sample data Test Set Recall:  0.0
Sample data Test Set FScore:  0.0


Sample data Train Set Confusion Matrix: 
[[855   1]
 [430   3]]
Sample data Train Set (Precision, Recall, Fscore) : 0.75 0.006928406466512702 0.013729977116704806


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  BernoulliNB()
[[9827    2]
 [ 570    3]]
Full Dataset Precision:  0.6
Full Dataset Recall:  0.005235602094240838
Full Dataset FScore:  0.010380622837370243


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Documentary
Number of 0s:  9342
Number of 1s:  1060
Shape of the Sampled DataFrame:  (3180, 44)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[476  42]
 [145 132]]
Sample data Test Set Precision:  0.7586206896551724
Sample data Test Set Recall:  0.47653429602888087
Sample data Test 

In [9]:
start = time.time()
df_test=gen_df_encode
for i in g_list:    
    df_test  = prediction(df_test,'tt_lemm',i,TfidfVectorizer, BernoulliNB(),  {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0]})
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\NB_Probability_Distribution_tt.pkl')
Final_data.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\Result_NB.pkl')
end = time.time()
total_time = end-start

print("Total Time taken:" + str(end-start))

Genre for prediction:  Horror
Number of 0s:  9456
Number of 1s:  946
Shape of the Sampled DataFrame:  (2838, 29)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[367 109]
 [ 70 164]]
Sample data Test Set Precision:  0.6007326007326007
Sample data Test Set Recall:  0.7008547008547008
Sample data Test Set FScore:  0.6469428007889546


Sample data Train Set Confusion Matrix: 
[[1253  163]
 [   9  703]]
Sample data Train Set (Precision, Recall, Fscore) : 0.8117782909930716 0.9873595505617978 0.891001267427123


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  BernoulliNB()
[[7583 1873]
 [  79  867]]
Full Dataset Precision:  0.3164233576642336
Full Dataset Recall:  0.9164904862579282
Full Dataset FScore:  0.4704286489419424


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  War
Number of 0s:  10100
Number o

Full Dataset Confusion Matrix  BernoulliNB()
[[9233  580]
 [ 136  453]]
Full Dataset Precision:  0.43852855759922554
Full Dataset Recall:  0.769100169779287
Full Dataset FScore:  0.558569667077682


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Crime
Number of 0s:  9176
Number of 1s:  1226
Shape of the Sampled DataFrame:  (3678, 38)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[542  54]
 [209 115]]
Sample data Test Set Precision:  0.6804733727810651
Sample data Test Set Recall:  0.3549382716049383
Sample data Test Set FScore:  0.46653144016227177


Sample data Train Set Confusion Matrix: 
[[1824   32]
 [ 140  762]]
Sample data Train Set (Precision, Recall, Fscore) : 0.9596977329974811 0.844789356984479 0.8985849056603774


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  BernoulliNB()
[[8557  619

C:\Users\ragha\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sample data Test Set Precision:  0.0
Sample data Test Set Recall:  0.0
Sample data Test Set FScore:  0.0


Sample data Train Set Confusion Matrix: 
[[37  0]
 [25  1]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 0.038461538461538464 0.07407407407407407


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  BernoulliNB()
[[10374     0]
 [   27     1]]
Full Dataset Precision:  1.0
Full Dataset Recall:  0.03571428571428571
Full Dataset FScore:  0.0689655172413793


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Mystery
Number of 0s:  9829
Number of 1s:  573
Shape of the Sampled DataFrame:  (1719, 43)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[290   0]
 [140   0]]


C:\Users\ragha\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sample data Test Set Precision:  0.0
Sample data Test Set Recall:  0.0
Sample data Test Set FScore:  0.0


Sample data Train Set Confusion Matrix: 
[[855   1]
 [429   4]]
Sample data Train Set (Precision, Recall, Fscore) : 0.8 0.009237875288683603 0.0182648401826484


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  BernoulliNB()
[[9828    1]
 [ 569    4]]
Full Dataset Precision:  0.8
Full Dataset Recall:  0.006980802792321117
Full Dataset FScore:  0.013840830449826988


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Documentary
Number of 0s:  9342
Number of 1s:  1060
Shape of the Sampled DataFrame:  (3180, 44)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[459  59]
 [140 137]]
Sample data Test Set Precision:  0.6989795918367347
Sample data Test Set Recall:  0.49458483754512633
Sample data Test Set